# Query Resolution using Fuzzy Matching and TF-IDF

This notebook demonstrates how to resolve new queries to a set of resolved queries using two main techniques:

- **Fuzzy String Matching** (using thefuzz library)
- **TF-IDF Vectorization with Cosine Similarity**

We will preprocess the data, apply both methods, and then combine the results to select the best match for each query.

In [1]:
import pandas as pd
from thefuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Import Required Libraries
We begin by importing the necessary libraries for data manipulation, fuzzy string matching, and text vectorization.

In [2]:
new_queries = pd.read_csv("new_queries.csv")
resolved_queries = pd.read_csv("resolved_queries.csv")

## Load Data
Read the new queries and resolved queries from their respective CSV files.

In [3]:
new_queries, resolved_queries

(                              Variation_Query  Matches_With_Query_ID
 0            Unabel to conect to the internet                      1
 1                   Can’t connect to internet                      1
 2                         Intenet not working                      1
 3                Payment failed while chekout                      2
 4   Payment did not go through during chckout                      2
 5                  Payment issue at check out                      2
 6    Application crashes when opening setings                      3
 7            App crash when going to settings                      3
 8            Settings cause the app to chrash                      3
 9               Forgot passwrd and cant reset                      4
 10        Forgotten password, unable to reset                      4
 11                  I can’t reset my password                      4
 12             Unable to uplod file to server                      5
 13        Can't upl

## Preview Data
Display the loaded dataframes to understand their structure and contents.

In [4]:
new_queries["Variation_Query"] = new_queries["Variation_Query"].str.lower().str.strip()
resolved_queries["Pre_Resolved_Query"] = resolved_queries["Pre_Resolved_Query"].str.lower().str.strip()

## Preprocess Queries
Convert queries to lowercase and strip whitespace to ensure consistent matching.

In [5]:
def get_all_fuzzy_scores(query, df, threshold=70):
    choices = df["Pre_Resolved_Query"].tolist()
    scores = {
        "ratio": process.extractOne(query, choices, scorer=fuzz.ratio),
        "partial_ratio": process.extractOne(query, choices, scorer=fuzz.partial_ratio),
        "token_sort_ratio": process.extractOne(query, choices, scorer=fuzz.token_sort_ratio),
        "token_set_ratio": process.extractOne(query, choices, scorer=fuzz.token_set_ratio),
    }
    best_method, best_result = max(scores.items(), key=lambda x: x[1][1])
    if best_result and best_result[1] >= threshold:
        # Find the Query_ID for the matched text
        qid = df.loc[df["Pre_Resolved_Query"] == best_result[0], "Query_ID"].values[0]
        return best_method, best_result[0], qid, best_result[1]
    return None, None, None, None

fuzzy_results = []
for uq in new_queries["Variation_Query"]:
    method, match, qid, score = get_all_fuzzy_scores(uq, resolved_queries)
    fuzzy_results.append((uq, method, match, qid, score))

fuzzy_df = pd.DataFrame(fuzzy_results, columns=[
    "Unresolved_Query", "Best_Method", "Fuzzy_Match", "Fuzzy_Query_ID", "Fuzzy_Score"
])

## Fuzzy String Matching
Apply various fuzzy string matching methods to find the closest resolved query for each new query.

In [6]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(
    new_queries["Variation_Query"].tolist() + resolved_queries["Pre_Resolved_Query"].tolist()
)

n_unresolved = len(new_queries)
unresolved_vecs = tfidf_matrix[:n_unresolved]
resolved_vecs = tfidf_matrix[n_unresolved:]

cosine_sim = cosine_similarity(unresolved_vecs, resolved_vecs)

tfidf_results = []
for i, uq in enumerate(new_queries["Variation_Query"]):
    best_idx = cosine_sim[i].argmax()
    best_score = cosine_sim[i][best_idx]
    matched_row = resolved_queries.iloc[best_idx]
    tfidf_results.append((uq, matched_row["Pre_Resolved_Query"], matched_row["Query_ID"], best_score))

tfidf_df = pd.DataFrame(tfidf_results, columns=[
    "Unresolved_Query", "TFIDF_Match", "TFIDF_Query_ID", "TFIDF_Score"
])

## TF-IDF Vectorization
Use TF-IDF to vectorize the queries and compute cosine similarity to find the best matches.

In [7]:
combined = pd.merge(fuzzy_df, tfidf_df, on="Unresolved_Query", how="inner")

## Combine Fuzzy and TF-IDF Results
Merge the results from both methods to prepare for final selection of the best match.

In [11]:
def pick_final(row, fuzzy_thresh=75, tfidf_thresh=0.65):
    if row["Fuzzy_Score"] and row["Fuzzy_Score"] >= fuzzy_thresh:
        return row["Fuzzy_Match"], row["Fuzzy_Query_ID"], f"Fuzzy-{row['Best_Method']}"
    elif row["TFIDF_Score"] >= tfidf_thresh:
        return row["TFIDF_Match"], row["TFIDF_Query_ID"], "TFIDF"
    else:
        return None, None, "No Match"

combined[["Final_Match", "Final_Query_ID", "Used_Method"]] = combined.apply(
    pick_final, axis=1, result_type="expand"
)

## Select Final Match
Apply thresholds to select the best match for each query, preferring fuzzy matches if above threshold, otherwise using TF-IDF if above its threshold.

In [12]:
combined[['Unresolved_Query', 'Fuzzy_Match', 'Used_Method']]

,Unresolved_Query,Fuzzy_Match,Used_Method
0,unabel to conect to the internet,unable to connect to the internet,Fuzzy-ratio
1,can’t connect to internet,unable to connect to the internet,Fuzzy-token_set_ratio
2,intenet not working,None,No Match
3,payment failed while chekout,payment failed during checkout,Fuzzy-ratio
4,payment did not go through during chckout,payment failed during checkout,No Match
5,payment issue at check out,payment failed during checkout,No Match
6,application crashes when opening setings,app crashes when opening settings,Fuzzy-partial_ratio
7,app crash when going to settings,app crashes when opening settings,Fuzzy-ratio
8,settings cause the app to chrash,None,No Match
9,forgot passwrd and cant reset,forgot password and unable to reset,Fuzzy-ratio


## Display Final Results
Show the unresolved queries, their best fuzzy match, and the method used for resolution.

## Conclusion
In this notebook, we explored two approaches—fuzzy string matching and TF-IDF vectorization with cosine similarity—to resolve new queries against a set of resolved queries. By combining the strengths of both methods, we improved the accuracy of query resolution. This approach can be further enhanced by tuning thresholds or incorporating more advanced NLP techniques for even better results.